## Part 1: Preprocessing

In [360]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [359]:
# Determine the number of unique values in each column.
attrition_df.shape

(1470, 27)

In [345]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [346]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [347]:
# Create a list of at least 10 column names to use as X data
X_columns = [
    'Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
    'JobInvolvement', 'JobSatisfaction', 'PerformanceRating',
    'RelationshipSatisfaction', 'WorkLifeBalance', 'YearsAtCompany'
]


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes


,0
Age,int64
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
JobInvolvement,int64
JobSatisfaction,int64
PerformanceRating,int64
RelationshipSatisfaction,int64
WorkLifeBalance,int64
YearsAtCompany,int64


In [348]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [349]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# all chosen columns in X_df are already numeric)

In [350]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [351]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = department_encoder.transform(y_train[['Department']])
department_test_encoded = department_encoder.transform(y_test[['Department']])

In [352]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[['Attrition']])
attrition_test_encoded = attrition_encoder.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [353]:
# Find the number of columns in the X training data
input_shape = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(60, 64, 1))

# Create at least two shared layers
shared_layer_1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer_2 = layers.Dense(32, activation='relu')(shared_layer_1)

In [354]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch = layers.Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
department_output = layers.Dense(department_train_encoded.shape[1], activation='softmax', name='department_output')(department_branch)


In [355]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = layers.Dense(16, activation='relu')(shared_layer_2)

# Create the output layer for Attrition
attrition_output = layers.Dense(attrition_train_encoded.shape[1], activation='sigmoid', name='attrition_output')(attrition_branch)


In [356]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', # This should not be using the binary cross-entropy loss.
                    'attrition_output': 'binary_crossentropy'},
              metrics=['accuracy'])

# Summarize the model
model.summary()

Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18            │ (None, 60, 64, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_82 (Dense)          │ (None, 60, 64, 64)     │            128 │ input_layer_18[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_83 (Dense)          │ (None, 60, 64, 32)     │          2,080 │ dense_82[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_84 (Dense)          │ (None, 60, 64, 16)     │            528 │ dense_83[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_85 (Dense)          │ (None, 60, 64, 16)     │            528 │ dense_83[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 60, 64, 3)      │             51 │ dense_84[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 60, 64, 2)      │             34 │ dense_85[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,349 (13.08 KB)

 Trainable params: 3,349 (13.08 KB)

 Non-trainable params: 0 (0.00 B)

In [357]:
# Train the model
model.fit(X_train_scaled,
          {'department_output': department_train_encoded, 'attrition_output': attrition_train_encoded},
          epochs=10,
          batch_size=32)


Epoch 1/10


ValueError: Input 0 of layer "functional_23" is incompatible with the layer: expected shape=(None, 60, 64, 1), found shape=(None, 10)

In [ ]:
# Evaluate the model with the testing data
loss, department_loss, attrition_loss, department_accuracy, attrition_accuracy = model.evaluate(
    X_test_scaled,
    {'department_output': department_test_encoded, 'attrition_output': attrition_test_encoded}
)

In [ ]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {department_accuracy:.4f}")
print(f"Attrition Accuracy: {attrition_accuracy:.4f}")

# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.